✍ В первой части данного юнита мы проанализируем построенную нами таблицу c рейтингами фильмов, а во второй — рассмотрим новые данные (заказы в интернет-магазине) и выполним задания по ним.

Результат нашего объединения таблиц с рейтингами вы можете найти [здесь](https://lms-cdn.skillfactory.ru/assets/courseware/v1/8e23b4ad7fb8d49f0b9c67be84dc440c/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/ratings_movies.zip) (csv-файл находится в zip-архиве — распакуйте архив, прежде чем продолжать работу!)

In [48]:
import pandas as pd

ratings_movies = pd.read_csv('data/movies/ratings_movies.csv')
ratings_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  100836 non-null  int64  
 1   userId      100836 non-null  int64  
 2   movieId     100836 non-null  int64  
 3   rating      100836 non-null  float64
 4   date        100836 non-null  object 
 5   title       100836 non-null  object 
 6   genres      100836 non-null  object 
dtypes: float64(1), int64(3), object(3)
memory usage: 5.4+ MB


Для решения задач нам понадобится выделить из признака title год выпуска фильма. 
Для этого напишем функцию get_year_release(arg).

In [2]:
import re 

def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

Данная функция основана на **регулярных выражениях** и использует модуль **re**, с которым мы пока не работали.

Модуль re предназначен для поиска шаблонов в тексте и встроен в язык, поэтому не нуждается в установке.

ДОПОЛНИТЕЛЬНО

При желании вы можете ознакомиться с регулярными выражениями по [ссылке](https://tproger.ru/translations/regular-expression-python/), однако это не является обязательным для выполнения заданий в этом юните.

Из модуля re нам понадобится только функция findall(), которая позволяет найти в строке все слова, удовлетворяющие шаблону. Мы находим в строке с названием фильма шаблон "(DDDD)" — четыре цифры, обёрнутых в скобки, что соответствует году выпуска фильма. Если такого шаблона не было найдено (год выпуска не указан), функция возвращает None (в таблице это будет помечено как пропуск).



In [51]:
# Задание 8.1
# Создайте в таблице новый признак year_release, который соответствует году выпуска фильма.
# У скольких фильмов не указан год их выпуска?


ratings_movies['year_release'] = ratings_movies['title'].apply(get_year_release)

ratings_movies[ratings_movies['year_release'].isna()]['year_release'].shape[0]

18

In [52]:
# Задание 8.2
# Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку зрителей?
# В качестве ответа запишите название этого фильма без указания года его выпуска.

fltr_Year = ratings_movies['year_release'] == 1999

film_1999 = ratings_movies[fltr_Year].groupby(by=['title']).mean(numeric_only=True).loc[:,['rating']]

film_1999.sort_values(by=['rating'])

,rating
title,
Bloodsport: The Dark Kumite (1999),0.5
Trippin' (1999),1.0
Chill Factor (1999),1.0
From Dusk Till Dawn 2: Texas Blood Money (1999),1.0
Simon Sez (1999),1.0
...,...
Mickey's Once Upon a Christmas (1999),5.0
On the Ropes (1999),5.0
Trailer Park Boys (1999),5.0


In [53]:
# Задание 8.3
# Какое сочетание жанров фильмов (genres), выпущенных в 2010 году, получило наименьшую среднюю оценку (rating)?
# Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).

fltr_Year = ratings_movies['year_release'] == 2010

film_2010 = ratings_movies[fltr_Year].groupby(by=['genres']).mean(numeric_only=True).loc[:,['rating']]

film_2010.sort_values(by=['rating'])


,rating
genres,
Action|Sci-Fi,1.000000
Action|Adventure|Horror,1.500000
Action|Drama|Fantasy,1.500000
Crime|Romance,1.500000
Adventure|Comedy|Fantasy,1.833333
...,...
Crime,4.750000
Adventure|Children|Comedy|Mystery,5.000000
Animation|Children|Mystery,5.000000


In [76]:
# Задание 8.4
# Какой пользователь (userId) посмотрел наибольшее количество различных (уникальных) комбинаций жанров (genres) фильмов? 
# В качестве ответа запишите идентификатор этого пользователя.

user_genres = ratings_movies.groupby(by=['userId']).nunique().head(1)

user_genres.sort_values('genres',ascending=False)

,Unnamed: 0,movieId,rating,date,title,genres,year_release
userId,,,,,,,
1,232,232,5,119,232,121,53


In [75]:
# Задание 8.5
# Найдите пользователя, который выставил наименьшее количество оценок, но его средняя оценка фильмам наибольшая.
# В качестве ответа укажите идентификатор этого пользователя.
# Чтобы рассчитать несколько параметров для каждого пользователя (количество оценок и среднюю оценку), 
# можно воспользоваться методом agg() на сгруппированных данных.

ratings_movies.groupby('userId')['rating'].agg(['count','mean']).sort_values(by=['count','mean'], ascending=[True, False]).head(1)

,count,mean
userId,,
53,20,5.0


In [83]:
# Задание 8.6
# Найдите сочетание жанров (genres) за 2018 год, которое имеет наибольший средний рейтинг (среднее по столбцу rating), 
# и при этом число выставленных ему оценок (количество значений в столбце rating) больше 10.
# Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).

l_ftr  = ratings_movies['year_release'] == 2018

rating_genres = ratings_movies[l_ftr].groupby('genres')['rating'].agg(['count','mean']).sort_values(by = ['mean'], ascending=False)

rating_genres[rating_genres['count'] > 10].head(1)


,count,mean
genres,,
Action|Adventure|Sci-Fi,14,3.928571


In [117]:
# Задание 8.7
# Добавьте в таблицу новый признак year_rating — год выставления оценки. 
# Создайте сводную таблицу, которая иллюстрирует зависимость среднего рейтинга фильма от года выставления оценки и жанра. 
# Выберите верные варианты ответа, исходя из построенной таблицы:

ratings_movies['year_rating'] = pd.to_datetime(ratings_movies['date'],yearfirst=True).dt.year

pivot_data = pd.pivot_table(ratings_movies,
                            values='rating',
                            index='genres',
                            columns='year_rating',
                            aggfunc='mean',
                            fill_value=0)


pivot_data[pivot_data.index == 'Comedy']

year_rating,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
genres,,,,,,,,,,,,,,,,,,,,,
Comedy,3.228571,3.409091,3.0,3.606061,3.141291,3.318408,3.198556,3.120066,3.356877,2.963325,...,3.273292,3.179825,3.232877,3.478477,3.489474,3.271429,3.106183,3.373431,2.852668,3.121296
